In [1]:
import numpy as np

## Métodos de Monte Carlo

Cuando tenemos preguntas sobre procesos, y tenemos modelos de mecanismo de un proceso, podemos usar [métodos de Monte Carlo](https://en.wikipedia.org/wiki/Monte_Carlo_method) para responderlas. Esto es extremadamente potente si ya sabes programar: hay muchas preguntas cuantitativas que son muy fáciles de formular y resolver como un experimento de Monte Carlo, pero muy difíciles de resolver de forma analítica.

### Ejemplo introductorio

Estamos jugando a cara o cruz con una moneda trucada. La moneda tiene la siguiente distribución de probabilidad:
```
cara: 0.6
cruz: 0.4
```

Si sale cruz ganas, si sale cara pierdes.

##### Pregunta 0

**¿Qué proporción de las veces ganas?**

Esta respuesta la podemos calcular trivialmente de forma analítica: `0.4`, que es simplemente la proporción de cruces esperada. Pero vamos a calcularla vía un experimento de Monte Carlo para romper mano. 

Llevamos a cabo "muchas" simulaciones de la tirada, y calculamos la proporción de cruces:

In [2]:
def estimate_win_proportion(num_trials: int) -> float:
    """
    cara: 0
    cruz: 1
    
    This simulates num_trials coin tosses and calculates num_cruces / total_tosses
    """
    trials = []
    
    for i in range(num_trials):
        if np.random.random() <= 0.4:
            trials.append(1)
        else:
            trials.append(0)
            
    return sum(trials) / len(trials)

for i in range(3):
    proportion_estimate = estimate_win_proportion(50000)
    print(proportion_estimate, "error: ", abs(0.4 - proportion_estimate))

0.39668 error:  0.003320000000000045
0.40504 error:  0.005039999999999989
0.40236 error:  0.0023599999999999732


Vemos que, efectivamente, la simulación de Monte Carlo nos da respuestas cercanas a la respuesta exacta, aunque en este caso sería más fácil calcular la proporción directamente.

##### Pregunta 1

Usamos la moneda en un juego con las siguientes reglas: la moneda se tira 2 veces, y dependiendo del número de cruces ganas (o pierdes) las siguientes cantidades:

| numero cruces | euros |
| ------------- | ----- |
| 0 | -2 |
| 1 | 0.5 |
| 2 | 1 |

**¿Cuál es [la media](https://en.wikipedia.org/wiki/Expected_value) de euros ganados (o perdidos) por partida si juegas a este juego con la moneda trucada?**

Esto también lo podemos calcular de forma analítica usando la [distribución binomial](https://en.wikipedia.org/wiki/Binomial_distribution), pero en este caso ya tenemos que saber más matemáticas:

$$
expected\_euros = -2 \cdot \binom{2}{0}0.6^{2} + 0.5 \cdot \binom{2}{1} 0.4 \cdot 0.6 + \binom{2}{2} 0.4 ^ 2
$$

In [3]:
exact_expected_euros = -2 * 1 * (0.6 ** 2) + 0.5 * 2 * (0.4 * 0.6) + 1 * 1 * (0.4 ** 2)
print(exact_expected_euros)

-0.31999999999999995


Si sabemos programar pero no usamos mucho las matemáticas en el día a día, igual es más fácil simularlo directamente que hacer el cálculo analítico:

In [4]:
def one_game() -> int:
    """
    This simulates a single game: a coin is tossed twice, the number of cruces is returned.
    """
    cruz_count = 0
    
    for i in range(2):
        if np.random.random() <= 0.4:
            cruz_count += 1
    return cruz_count

def average_euros(num_trials: int) -> float:
    """
    This simulates num_trials games and calculates total_euros / num_trials
    """
    
    euros = []
    for i in range(num_trials):
        cruz_count = one_game()
        if cruz_count == 0:
            euros.append(-2)
        elif cruz_count == 1:
            euros.append(0.5)
        else:
            euros.append(1)
            
    return sum(euros) / len(euros)
    
    
for i in range(3):
    estimated_expected_euros = average_euros(50000)
    print(estimated_expected_euros, "error: ", abs(estimated_expected_euros - exact_expected_euros))
    

-0.3302 error:  0.010200000000000042
-0.31169 error:  0.008309999999999929
-0.31771 error:  0.0022899999999999587


### Preguntas de calentamiento

Consideramos un juego con 2 dados de 6 caras: el dado `a` es justo, el dado `b` tiene un sesgo que hace que aparezca más veces el 6 que los otros números, la tabla siguiente da las probabilidades:

| dado | 1 | 2 | 3 | 4 | 5 | 6 |
| ---- | - | - | - | - | - | - |
| a | 1/6 | 1/6 | 1/6 | 1/6 | 1/6 | 1/6 |
| b | 1/12 | 1/12 | 1/12 | 1/12 | 1/12 | 7/12 |

Consideramos la suma resultante de tirar `a` y `b` y sumarlos.

#### Pregunta 1

Jugamos a un juego en el que el jugador gana si saca más de un 8 y pierde si saca menos.

**¿Qué proporción de las veces esperas ganar si juegas a este juego con los dados a y b?**

#### Pregunta 2

Pintar histogramas del resultado de las sumas de estos dados cuando se ejecutan simulaciones de Monte Carlo de 10, 100, y 10000 tiradas.

Pensar en lo que quiere decir esto:
* ¿Cuanto se parecen entre sí dos histogramas de 10 tiradas distintos?
* ¿Cuanto se parecen entre sí dos histogramas de 10000 tiradas distintos?
* Hemos caracterizado los dados `a` y `b` como tablas de valores y sus probabilidades. Pinta una tabla que haga lo mismo para los valores de la suma de a y b:

| 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 |
| - | - | - | - | - | - | - | - | -- | -- | -- |
| ? | ? | ? | ? | ? | ? | ? | ? | ? | ? | ? |